## Preparation

#### Import libraries needed.

In [2]:
import pandas as pd
import re 
import xlrd # to read excel
import nltk # NLP toolkit
import matplotlib.pyplot as plt # for visualization
import string # for handling string
import seaborn as sns
import spacy

#### Access the location of origin file by declaring the path

In [3]:
# give the location 
data_path = '/Users/cathzzr2/Desktop/dataset/'
data_filename = 'dataset.xlsx'
# default path to the origin_file
summary = pd.read_excel(data_path + data_filename,header=0) # (default: 0)

#### 1.1 Basic Properties - size, head, tail, etc. 

In [ ]:
print(summary.shape) # size of the excel: (row, colume)
print(summary.columns) # index of columns
print(summary.head(3)) # preview top 3 rows (default: 5)
print(summary.tail(3)) # preview top 3 rows (default: 5)
print(summary.describe) # only integer variables are shown
print(summary.nunique)
print(summary['Authors'].unique())

#### 2.1 Cleaning the Data - remove null cells, convert to lowercases, remove punctuations

In [ ]:
# number of null cells in each columns
print(summary.isnull().sum()) 
# drop unnecessary index and store it to a new file
new = summary.drop(['Date', 'UID', 'PDF URL', 'Cyber_Risk', 'Not_Cyber'], axis=1) 
# Lowercase the summary
new['summary_in_lowercase']=new['Summary'].apply(lambda x: x.lower())
new['summary_without_punkt']=new['summary_in_lowercase'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
print(new.head(1))

#### 2.2 Cleaning the Data - remove stopwords

In [ ]:
from nltk.corpus import stopwords
stoplist = set(stopwords.words("english"))
nltk.download('punkt')

## Exploratory Data Analysis

#### 3.1 Tokenization

In [7]:
from nltk.tokenize import word_tokenize 

# tokenization
def preprocess(text):
  formatted_text = text.lower()
  tokens = []
  for token in nltk.word_tokenize(formatted_text):
    tokens.append(token)
  tokens = [word for word in tokens if word not in stoplist and word not in string.punctuation]
  formatted_text = ' '.join(element for element in tokens)

  return formatted_text
# remove stop words
new['summary_without_stopw']=new['summary_without_punkt'].apply(lambda x: preprocess(x))

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
new['lemmatized']=new['summary_without_stopw'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))


#### 4.1 Word Frequency - Document Term Matrix

In [ ]:
# Creating Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(analyzer='word')
data=cv.fit_transform(new['lemmatized'])
new_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
new_dtm.index=new.index
new_dtm.head(30) # customize rows

#### 4.2.1 Word Frequency - Visualization - Word Cloud 

In [ ]:
from wordcloud import WordCloud
from textwrap import wrap
# Function for generating word clouds
def generate_wordcloud(data,title):
  wc = WordCloud(width=400, height=330, max_words=150,colormap="Dark2").generate_from_frequencies(data)
  plt.figure(figsize=(10,8))
  plt.imshow(wc, interpolation='bilinear')
  plt.axis("off")
  plt.title('\n'.join(wrap(title,60)),fontsize=13)
  plt.show()
  
# Transposing document term matrix
new_dtm=new_dtm.transpose()

# Plotting word cloud for each product
for index,product in enumerate(new_dtm.columns):
  generate_wordcloud(new_dtm[product].sort_values(ascending=False),product)

#### 4.2.2 Word Frequency - Visualization - Plot with MATLAB